# Performing OCR with SVMs

Image processing is a difficult task for many types of machine learning algorithms. The relationships linking patterns of pixels to higher concepts are extremely complex and hard to define. It's easy for a human to recognize a face, a cat, or the letter A, but defining these patterns in strict rules is difficult. Furthermore, image data is usually very noisy. Depending on lighting, orientation or positioning it can be very hard to tell.

SVMs are well-suited for tackling image data. Capable of learning complex patterns without being overly sensitive to noise, they are able to recognize visual patterns with a high degree of accuracy.

Moreover, the key weakness of SVMs - The Black Box model representation - is less critical for image processing. If it can detect the difference between a cat and a dog, how it does it doesn't matter as much.

## Collecting Data

When OCR software first processes a document, it divides the paper into a matrix such that each cell in the grid contains a single glyph. Next it will try to match the glyph to all characters it recognizes. Finally it'll all be combined back together for words, which optionally could be spell-checked against a dictionary in the document's language.

We'll be using data from the UCI Machine Learning Data Repository, containing 20,000 examples of 26 english alphabet capital letters printed using 20 different randomly reshaped and distorted black and white fonts.

## Exploring and preparing the data

In [21]:
letters <- read.csv("C:/Users/Yuchi/Downloads/letter-recognition.csv")

In [22]:
str(letters)

'data.frame':	20000 obs. of  17 variables:
 $ Letter: Factor w/ 26 levels "A","B","C","D",..: 20 9 4 14 7 19 2 1 10 13 ...
 $ X.box : int  2 5 4 7 2 4 4 1 2 11 ...
 $ Y.box : int  8 12 11 11 1 11 2 1 2 15 ...
 $ width : int  3 3 6 6 3 5 5 3 4 13 ...
 $ high  : int  5 7 8 6 1 8 4 2 4 9 ...
 $ onpix : int  1 2 6 3 1 3 4 1 2 7 ...
 $ X.bar : int  8 10 10 5 8 8 8 8 10 13 ...
 $ Y.bar : int  13 5 6 9 6 8 7 2 6 2 ...
 $ x2bar : int  0 5 2 4 6 6 6 2 2 6 ...
 $ y2bar : int  6 4 6 6 6 9 6 2 6 2 ...
 $ xybar : int  6 13 10 4 6 5 7 8 12 12 ...
 $ x2ybr : int  10 3 3 4 5 6 6 2 4 1 ...
 $ xy2br : int  8 9 7 10 9 6 6 8 8 9 ...
 $ X.ege : int  0 2 3 6 1 0 2 1 1 8 ...
 $ xegvy : int  8 8 7 10 7 8 8 6 6 1 ...
 $ Y.ege : int  0 4 3 2 5 9 7 2 1 1 ...
 $ yegvx : int  8 10 9 8 10 7 10 7 7 8 ...


NOTE: SVM Learners require all features to be numeric, and moreover, that each feature is scaled to a fairly small interval. Every feature in this case is an int, so we don't have to convert anything. On the other hand some of the ranges are fairly wide. This suggest we probably need to normalize or standardize the data. However the package we'll be using for this will do this for us automatically.

We would usually create a random section fro training and testing data sets, but in this data set it has already been randomized and would suggest using the first 16,000 for building the model and the next 4,000 for testing.

In [26]:
letters_train <- letters[1:16000,]

In [27]:
letters_test <- letters[16001:20000,]

## Training a model on the data

There are a lot of SVM packages to use. The **e1071** package provides an R interface to the LIBSVM library, a widely-used open source SVM program. 
If you're already invested in the SVMlight algorithm, the **klaR** package provides functions to work with this SVM implementation directely from R.
Finally, if you're starting from scratch, it might be worth starting with the functions in the **kernlab** package. This was developed natively within R rather than C or C++ which alows it to be easily customized. Most importantly, **kernlab** can be used with the **caret** package, which allows SVM models to be trained and evaluated using a variety of automated methods.

In [7]:
install.packages("kernlab", repos = "http://cran.us.r-project.org")

Installing package into 'C:/Users/yuchi/Documents/R/win-library/3.4'
(as 'lib' is unspecified)


package 'kernlab' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\yuchi\AppData\Local\Temp\RtmpEdmCxt\downloaded_packages


In [8]:
library(kernlab)

We are calling the *ksvm* function on the training data and specify the linear(that is,vanilla) kernal using the *vanilladot* option.

In [28]:
letter_classifier <- ksvm(Letter ~ ., data = letters_train, kernel = "vanilladot")

 Setting default kernel parameters  


In [14]:
letter_classifier

Support Vector Machine object of class "ksvm" 

SV type: C-svc  (classification) 
 parameter : cost C = 1 

Linear (vanilla) kernel function. 

Number of Support Vectors : 7039 

Objective Function Value : -14.1747 -20.007 -23.5629 -6.2007 -7.5523 -32.7693 -49.9788 -18.1824 -62.111 -32.7284 -16.221 -32.2839 -28.9776 -51.2192 -13.276 -35.6223 -30.8612 -16.5255 -14.681 -32.7472 -30.3216 -7.7959 -11.814 -32.3455 -13.126 -9.2693 -153.165 -52.9678 -76.7743 -119.2073 -165.4435 -54.6247 -41.9818 -67.2686 -25.1959 -27.6368 -26.41 -35.5578 -41.26 -122.1636 -187.9174 -222.0861 -21.4765 -10.3749 -56.3682 -12.2279 -49.4902 -9.3371 -19.2099 -11.1776 -100.2194 -29.14 -238.0507 -77.1985 -8.334 -4.5309 -139.8544 -80.8849 -20.3643 -13.0243 -82.515 -14.5037 -26.7516 -18.5709 -23.9512 -27.3041 -53.273 -11.4773 -5.1202 -13.9501 -4.4981 -3.5754 -8.4912 -40.971 -49.8188 -190.0265 -43.8604 -44.868 -45.258 -13.5555 -17.767 -87.4103 -107.1064 -37.025 -30.713 -112.3208 -32.9635 -27.2966 -35.5832 -17.8585 -5.139

This info tells us very little about how well the model will perform in the real world. We'll need to examine it's performance on the testing dataset to know whether it generalizes well to unseen data.

## Evaluating model performance

The *predict()* function allows us to use the letter classification model to make predictions on the testing dataset.

In [29]:
letter_predictions <- predict(letter_classifier, letters_test)

Because we didn't specify the type parameter, the default type = "response" was used. This returns a vector containing a predicted letter for each row of values in the testing data. Using the *Head()* function, we can see that the first six predicted values.

In [30]:
head(letter_predictions)

[1] U N V X N H
Levels: A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

In order to examine how well our classifier performed, we need to compare the predicted letter to the true letter in the testing dataset. We'll use *table()* function for this.

In [31]:
table(letter_predictions, letters_test$Letter)

                  
letter_predictions   A   B   C   D   E   F   G   H   I   J   K   L   M   N   O
                 A 144   0   0   0   0   0   0   0   0   1   0   0   1   2   2
                 B   0 121   0   5   2   0   1   2   0   0   1   0   1   0   0
                 C   0   0 120   0   4   0  10   2   2   0   1   3   0   0   2
                 D   2   2   0 156   0   1   3  10   4   3   4   3   0   5   5
                 E   0   0   5   0 127   3   1   1   0   0   3   4   0   0   0
                 F   0   0   0   0   0 138   2   2   6   0   0   0   0   0   0
                 G   1   1   2   1   9   2 123   2   0   0   1   2   1   0   1
                 H   0   0   0   1   0   1   0 102   0   2   3   2   3   4  20
                 I   0   1   0   0   0   1   0   0 141   8   0   0   0   0   0
                 J   0   1   0   0   0   1   0   2   5 128   0   0   0   0   1
                 K   1   1   9   0   0   0   2   5   0   0 118   0   0   2   0
                 L   0   0   0   

The diagonal values indicate the total number of records where the predicted letter matches the true value. Similarly, the number of mistakes is also listed. For example, the value 5 in the row B and column D indicates there were 5 cases where the letter D was mistaked as a B.

We can simplify our evaluation by instead calculating overall accuracy. This considers only whether the prediction was correct or incorrect and ignores the type of error. The following command returns a vector of TRUE or FALSE values indicating whether the model's predicted letter agrees with the actual letter.

In [33]:
agreement <- letter_predictions == letters_test$Letter

In [34]:
table(agreement)

agreement
FALSE  TRUE 
  643  3357 

In [35]:
prop.table(table(agreement))

agreement
  FALSE    TRUE 
0.16075 0.83925 

About an 84% accuracy.

## Improving Model Performance

Our previous model used the simple linear kernal function. By using a more complex kernal, we can map the data into a higher dimensional space and potentially obtain a better model fit. 

It's pretty challenging to choose which one from the many options. A popular convention is to begin with the Gaussian RBF kernal, which has been shown to perform well for many types of data.

In [36]:
letter_classifier_rbf <- ksvm(Letter ~ ., data = letters_train, kernel = "rbfdot")

Making predictions similar to before.

In [37]:
letter_predictions_rbf <- predict(letter_classifier_rbf, letters_test)

Comparing accuracy to our linear SVM

In [38]:
agreement_rbf <- letter_predictions_rbf == letters_test$Letter

In [39]:
table(agreement_rbf)

agreement_rbf
FALSE  TRUE 
  281  3719 

In [40]:
prop.table(table(agreement_rbf))

agreement_rbf
  FALSE    TRUE 
0.07025 0.92975 

By just changing the kernel function, we were able to increase the accuracy of our character recognition model from 84% to 93%. If this accuracy is still not good enough for the OCR, other kernels could be tested or the COST could be varied to modify the width of the decision boundary.

In [41]:
table(letter_predictions_rbf, letters_test$Letter)

                      
letter_predictions_rbf   A   B   C   D   E   F   G   H   I   J   K   L   M   N
                     A 151   0   0   0   0   0   0   0   0   0   0   0   0   0
                     B   0 127   0   3   0   1   0   2   0   0   0   1   2   1
                     C   0   0 132   0   3   0   1   0   2   0   0   1   0   0
                     D   1   1   0 161   0   0   2   9   2   3   1   0   0   1
                     E   0   0   3   0 137   2   0   0   0   1   0   4   0   0
                     F   0   0   0   0   0 148   0   0   3   0   0   0   0   0
                     G   0   0   2   0   8   0 155   2   0   0   0   2   2   0
                     H   0   1   0   1   0   0   1 124   0   1   2   1   1   3
                     I   0   0   0   0   0   0   0   0 151   3   0   0   0   0
                     J   0   0   0   0   0   0   0   0   3 136   0   0   0   0
                     K   0   0   1   0   0   0   0   5   0   0 132   0   0   1
                     L   0   

Looking deeper, it looks like the biggest similar letters causing issues are D's getting mistaken for H's and especially F's being mistaken for P's.